In [ ]:
import tensorflow as tf
import numpy as np

### Experimentation with tf.nn.conv2d

##### Input tensor shape is (batch_size, in_height, in_width, in_channels), in_channels representing for e.g. RGB

In [ ]:
x_in = np.array([[
  [[2], [1], [2], [0], [1]],
  [[1], [3], [2], [2], [3]],
  [[1], [1], [3], [3], [0]],
  [[2], [2], [0], [1], [1]],
  [[0], [0], [3], [1], [2]],
  [[0], [0], [3], [1], [2]],
  [[0], [0], [3], [1], [2]]
]])

In [ ]:
x_in.shape

In [ ]:
x_in.shape[3]

##### Kernel/filter tensor is of input shape (filter_height, filter_width, in_channels, out_channels), out_channels represents how X many of these filters applied to the input (or neurons units)

In [ ]:
kernel_in = np.array([
 [ [[2, 0.1, 0.9]], [[3, 0.2, 0.8]], [[4, 0.7, 0.6]] ],
 [ [[0, 0.3, 0.6]], [[1, 0.4, 0.8]], [[2, 0.2, 0.4]] ] ])

In [ ]:
kernel_in

In [ ]:
kernel_in.shape

In [ ]:
kernel_in = np.array([
 [ [[2, 0.1]], [[3, 0.2]], [[4, 0.7]] ],
 [ [[0, 0.3]],[[1, 0.4]], [[2, 0.7]] ] ])

In [ ]:
kernel_in.shape

##### Output of applied convolution layer

In [ ]:
conv2d_layer = tf.nn.conv2d(input=x_in, filters=kernel_in, strides=1, padding='SAME')

In [ ]:
conv2d_layer

#### Experiment with Keras layers for insight

In [ ]:
model = tf.keras.models.Sequential([
    # First convolution layer
    tf.keras.layers.Conv2D(64, (2,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPool2D(2,2),
    # Second convolution layer
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # Flatten layer to input into DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Output binary neuron
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

### Build Custom Conv2D Layer Class

#### Experiment

In [ ]:
kernel = (3, 3)
in_channels = 1
out_channels = 32 # nr of filters applied, equiv. to nr. of neurons

In [ ]:
filter_weights_shape = kernel + (in_channels, out_channels)

In [ ]:
filter_weights_shape

In [ ]:
weights = tf.Variable(
    tf.random.normal(filter_weights_shape, stddev=0.1),
    trainable= True,
    dtype=tf.float32,
    name="conv2d_filters"
    )
weights

In [ ]:
x_in.shape

In [ ]:
conv2d_layer = tf.nn.conv2d(input=x_in, filters=weights, strides=(1,1), padding="VALID")

#### Implement Class Layer

In [ ]:
class Conv2D(tf.Module):
    """
        2D Convolution Layer
    """
    
    def __init__(self,
                out_channels: int, # aka nr. filters
                kernel: int, # the h*w of filter, can be a tuple but for now let's set it to a sqaure
                stride: int, # the h_steps*w_steps of filter as it moves along input, set to a square for now
                use_bias: bool = True,
                name = None
                ):
    
        super(Conv2D, self).__init__(name)
        
        self.out_channels = out_channels
        self.kernel: Tuple[int, int] = (kernel, kernel) # h*w of kernel
        self.stride: Tuple[int, int] = (stride, stride) # h_step*w_step of kernel movement
        self.use_bias = use_bias
        
        self.is_built: bool = False
            
        self.W: Optional[tf.Variable] = None
        self.b: Optional[tf.Variable] = None
            
    
    def __call__(self, x_in: tf.Tensor) -> tf.Tensor:
        """
            Build tensor on the first call.
            Calculate output by 2D convolution.
            
            :param x_in: input tensor of shape (batch_size, in_height, in_width, in_channels)
            
        """
        
        if not self.is_built: # initialize weights
            in_channels = x_in.shape[3]
            filter_weights_shape = self.kernel + (in_channels, self.out_channels)
            
            self.W = tf.Variable(tf.initializers.GlorotUniform()(filter_weights_shape),
                                 trainable = True,
                                 dtype = tf.float32,
                                 name = "conv2d_filters"
                                )
            print(f"weights is: {self.W}")
            
            if self.use_bias:
                self.b = tf.Variable(tf.initializers.GlorotUniform()((self.out_channels,)),
                                    trainable = True,
                                     dtype = tf.float32,
                                     name = "conv2d_bias"
                                    )
                print(f"bias is: {self.b}")
                
            self.is_built = True # first time weights and bias has been set, this will prevent it from being reset during backprop
        
        conv2d_layer = tf.nn.conv2d(input=x_in, filters=self.W, strides=self.stride, padding="VALID")
        print(f"conv2d output is: {conv2d_layer}")
              
        if self.use_bias:
            return tf.add(
                conv2d_layer,
                self.b,
                name = "con2d_add_bias")
        
        else:
            return conv2d_layer

In [ ]:
conv2d_layer = Conv2D(out_channels=3, kernel=2, stride=1)

In [ ]:
conv2d_layer.__call__(x_in=x_in)